<a href="https://colab.research.google.com/github/gacerioni/redis-workshop-json-search-vs/blob/master/redis-workshop-vector-similarity-search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Workshop - Redis como VectorDB - VSS e LLM (soon)

![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)


Bem-vind[ao]s ao Workshop! Vamos ter uma experiência hands-on sobre alguns temas centrais do Redis, bem além do Caching.


Para uma experiência premium, como a que eu quero que vocês tenham, recomendo MUITO utilizar o Redis Insight (App ou Web) pra apoiar na visualização dos dados.

https://redis.com/redis-enterprise/redis-insight/

## Objetivos do Workshop

Dessa vez, vamos direto ao ponto. Para pegar o fio da meada, passando pela introdução, veja este outro notebook [aqui](https://colab.research.google.com/github/gacerioni/redis-workshop-json-search-vs/blob/master/redis-workshop-vector-similarity-search.ipynb).

Este notebook irá fazer explorar o Redis como um VectorDB.

Vamos gerar os embeddings utilizando um modelo de SentenceTransformer da HuggingFace, o [all-MiniLM-L6-v2](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2).

Depois, vamos explorar as aplicações e casos de uso relacionados ao mundo dos Vetores.

Este notebook irá servir de base para casos de uso mais avançados para o mundo de LLM: RAG, Semantic Caching, Recommendation Systems, etc.

Espero que gostem! 🖖

# Setup Rápido

In [1]:
# Instale o Redis client e tambem o Hugging Face sentence transformers, pois vamos gerar os vetores aqui mesmo
!pip install -q redis sentence_transformers

# E instalar a CLI, via redis-tools, que inclui a famosa redis-cli
!apt-get update
!apt-get install -y redis-tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 56.7 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,083 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-dri

### Conectando com o Redis server

In [2]:
import os

# Coloque aqui os dados do seu DB do Redis Cloud
REDIS_HOST="redis-19581.c308.sa-east-1-1.ec2.redns.redis-cloud.com"
REDIS_PORT=19581
REDIS_PASSWORD="nhtuquVSLbh2kUt2I86z5QwGu3KrcaYx"

# Caso o SSL esteja ativo pro endpoint, adicione --tls
# Recomendo não misturar lé com cré aqui, visto que não vamos ter nenhuma informação sensível passando pelo fio.
if REDIS_PASSWORD!="":
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT} -a {REDIS_PASSWORD} --no-auth-warning"
else:
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT}"

# Caso o SSL esteja ativo pro endpoint, use rediss:// como o URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"
INDEX_NAME = f"qna:idx"

# Teste a Redis connection
!redis-cli $REDIS_CONN PING

PONG


In [4]:
# Testando via Python (redis-py)
import redis
redis = redis.Redis(
  host=REDIS_HOST,
  port=REDIS_PORT,
  password=REDIS_PASSWORD)
redis.ping()

True

In [5]:
# Boring part - config do pandas e deps, nao se preocupem!
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm
from redis import Redis
from redis.commands.search.field import (
    NumericField,
    TagField,
    TextField,
    VectorField,
)
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query


tqdm.pandas()



/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


# Iniciando os trabalhos

Vamos começar com o embedding. Ou seja, como gerar esses tais vetores a partir de informações não estruturadas.

### Embedding - Hora de trabalhar com o generation model

Aqui começa o samba. Vamos usar o `sentence-transformers/all-MiniLM-L6-v2` da HuggingFace para gerar nosso embedding de vetores.

Vai ser bacana, e vai abrir a mente sobre usos fora do mundo de Machine Learning... como buscas performáticas por features e aproximações semânticas.

Este modelo possui 384 dimensões, o que já é mais do que suficiente para a nossa demo, considerando os tokens e tudo mais.

---
Vale dizer que existem modelos pra quase tudo na nossa vida, inclusive os famosos Transformers, que podem ser de imagem, video, e áudio também. Neste caso, vamos com um SentenceTransformer, que é de **texto**.

---


Vamos começar simples: vamos carregar o **SentenceTransformer**:



In [7]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

#### Carregando a massa de dados - Download de 12k+ tweets

Vamos carregar uma massa de dados de 12000 tweets no Redis. Essa massa será o objeto do nosso estudo, com vetores.

In [8]:
# baixando o csv localmente
!wget https://raw.githubusercontent.com/antonum/Redis-VSS-Streamlit/main/Labelled_Tweets.csv

--2024-05-28 13:46:28--  https://raw.githubusercontent.com/antonum/Redis-VSS-Streamlit/main/Labelled_Tweets.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2486081 (2.4M) [text/plain]
Saving to: ‘Labelled_Tweets.csv’

Labelled_Tweets.csv 100%[===================>]   2.37M  --.-KB/s    in 0.01s   

2024-05-28 13:46:29 (221 MB/s) - ‘Labelled_Tweets.csv’ saved [2486081/2486081]



In [9]:
# carregando os dados via pd
df = pd.read_csv('Labelled_Tweets.csv').drop(columns=['created_at','score'])
# opcional, pra gente segurar o reggae caso estejamos travados no cap de 30MB do Free Tier.
#df=df.head(3000) #trim dataframe to fit results into 30MB Redis database
df


,id,full_text
0,1,@KennyDegu very very little volume. With $10T ...
1,2,#ES_F achieved Target 2780 closing above 50% #...
2,3,RT @KimbleCharting: Silver/Gold indicator crea...
3,4,@Issaquahfunds Hedged our $MSFT position into ...
4,5,RT @zipillinois: 3 Surprisingly Controversial ...
...,...,...
12415,12587,RT @PeterLBrandt: $SPX $ES_F \r\nFollowing thi...
12416,12588,RT @vieiraUAE: Fearless Alex Vieira Calls Best...
12417,12589,$spy $spx $qqq $ndx #nyse going from poking th...
12418,12590,RT @DavidScottAdams: On watch tomorrow // Pt. ...


### Gerando os Embeddings (o Vetor em si, finalmente)

Vamos gerar o "vector embedding" para o nosso dataframe.
Este passo pode levar 2-4 minutinhos, pois a GPU aqui é de demo rsrs


Notem que eu optei por utilizar o campo `full_text` para gerar os embeddings. Entretanto, em breve você irá ficar craque em decidir o que usar para gerar essa "impressão digital" da Entidade. Um filme, por exemplo, poderia usar uma concatenação do sumário, gênero, resenhas, paleta de cores, etc.


In [10]:
def text_to_embedding(text):
  return model.encode(text).astype(np.float32).tobytes()

#generate vector embeddings
df["text_embedding"] = df["full_text"].progress_apply(text_to_embedding)
df.head()

  0%|          | 0/12420 [00:00<?, ?it/s]

,id,full_text,text_embedding
0,1,@KennyDegu very very little volume. With $10T ...,b'\'\x92\x81\xbd\rh\x8b\xbd|\xdf\xe4\xbc\xbb\x...
1,2,#ES_F achieved Target 2780 closing above 50% #...,b'O\x1b\x02\xbd\x15~/\xbd\x8ez\xb1\xbcY\x99\xd...
2,3,RT @KimbleCharting: Silver/Gold indicator crea...,b'\x0e\xaa\xa3\xbdi}\x10\xbd\xc7\xe8\xb9=)\x08...
3,4,@Issaquahfunds Hedged our $MSFT position into ...,b'\xbd\x7f\xd1\xbc\xc1\n`\xbd59 =\xe1\xc0\xef=...
4,5,RT @zipillinois: 3 Surprisingly Controversial ...,b'\xc7\r\x1e\xbdZ\\\xd4\xbcS/\xa1\xbc\xe7q7=\x...


### Criando as Helper Functions que iremos utilizar

Aqui não tem segredo. Vamos desenhar as functions que irão nos apoiar nessa reta final. Já temos os embeddings, agora vamos guardar no Redis a entidade completa + embeddings, o que é uma boa prática.

Aqui, nós vamos:
- Salvar os dados como HASH no Redis;
- Criar o RediSearch Index corretamente, explicando pro Redis o que é aquele embedding curioso que estamos adicionando no bolo.

In [11]:
def load_dataframe(redis, df, key_prefix="tweet", id_column="id", pipe_size=100):
    records = df.to_dict(orient="records")
    pipe = redis.pipeline(transaction=False)
    i=1
    for record in tqdm(records):
        i=i+1
        key = f"{key_prefix}:{record[id_column]}"
        pipe.hset(key, mapping=record)
        if (i+1) % pipe_size == 0:
          res=pipe.execute()
    pipe.execute()

def create_redis_index(redis, idxname="tweet:idx"):
  try:
    redis.ft(idxname).dropindex()
  except:
    print("no index found")

  # Create an index
  indexDefinition = IndexDefinition(
      prefix=["tweet:"],
      index_type=IndexType.HASH,
  )

  redis.ft(idxname).create_index(
      (
          TextField("full_text", no_stem=False, sortable=False),
          VectorField("text_embedding", "HNSW", {  "TYPE": "FLOAT32",
                                                    "DIM": 384,
                                                    "DISTANCE_METRIC": "COSINE",
                                                  })
      ),
      definition=indexDefinition
  )



### Criando o index e carregando os dados para o Redis

In [12]:
# Opcional - dar um reset brabo no Redis - vai perder tudo la
redis.flushdb()

# criando o Index
create_redis_index(redis)

# load data from Dataframe to Redis HASH
# carregando os dados do CSV, os tweets, para o Redis (como HASH, mas poderia ser JSON. Life's good.)
load_dataframe(redis,df,key_prefix="tweet", pipe_size=100)


no index found


  0%|          | 0/12420 [00:00<?, ?it/s]

In [13]:
# De uma olhada em como o dado está ficando no seu Redis
# o text_embedding ta la, junto dos metadados do Tweet 1001, e sera a nossa impressao digital daqui pra frente
!redis-cli $REDIS_CONN hgetall "tweet:1001"

1) "text_embedding"
2) "\x87\x1d\x03:\xe2b\xbd\xbcB\xb0\x94\xbc.K)<\xf3T\x94;%>\x11\xbd\xa83\r=\xfe\x97\x7f=\x91X\x88<xUR\xbd\xc5\xf6h\xbc\x82\xb5\xd2\xbb\x10;\t\xbd\xecm#<\xe1\xd4C=\x88\xc8\"\xbd\x16\xa3-\xbd+^\t\xbdt\x00\xfd\xbc\xafGE\xbdo\xadZ\xbc\\\xae=\xbdH<\x99<\xfbb\xce\xbc\t`\xab<HDT\xbchM\xd7\xbc\xdb\xf0\x82\xbb\x90\xb0(=)v\x8a\xbc\xcc\xa8M\xbdh,P=\xfeR\x86<\x8aj\xfa<??\xb5<\x83[\xee\xbb\xce\x8d\x90=\x10W8=P\x81\x82\xbd\xa3\xfb&<\xee-:=U\xe6\xe4\xbd\xa9\x1d\x11\xbc\xd3\xb7\xac\xbd\xb6Y<\xbbZ\xc9\xcc\xbb\xdb9b\xbcI\xe3\x90=2\x9f\xa7=\xe67\x0f=\x17\xbb\xbb\xbbQ\xad\x17=\xbe\xaeE\xbd\xe0\x8e\x90={\xd7\x02\xbd\xd6\x01\x9f\xbc\xb8\xc3e;\x02\xa5\xd8\xbcs{f<\x9fx\xaf\xbd\xe6\xc4\x89\xbcXo\x03=\x86B\x8e\xbc\xfa\x90G;\a\xe7\x97\xba\xc2\xc1\x9a=\x17\xf7B\xbd\xbb\r\xca<\xc3\xf2G\xbb\xc7\x0e\x86<\x9br\x84<\xcb\xf7^\xbd3\xc9\xbe\xbdaK5\xbc%\xe09\xbd\xbf\xd7==5y\xb3=\xb2\x10\xb4\xbdq\xfc\x11\xbdG\x87O\xbc\xceo<\xbdz\x87\x80\xbc\x1f\xa9\"\xbdU) \xbd.\xfa\x85\xba\xc9y\x95=\xf5\xdf\xa6<+wF<\b\

## Consulte o Redis, como um Banco de Dados primário agora!

[Always-on Streamlit app](https://antonum-redis-vss-streamlit-streamlit-app-p4z5th.streamlit.app/)


Vamos experimentar queries como:
“Oil”, “Oil Reserves”, “Fossil fuels”


Note que as buscas Full-Text que vão pelo "léxico apenas" são exauridas rapidamente... talvez abaixo do que você espera de uma Search Engine (como o ELK, por exemplo).

Entretanto, o Vector está com muitas (e boas) dimensões, pra gente trazer mais informação relevante para o nosso contexto aqui. Você que pode decidir depois como que o Vector Search está funcionando debaixo do capô.

In [14]:
user_query="fossil fuels"
# use tbm "oil reserve", "fossil fuels"

In [15]:
# usando a Full-Text contra o nosso Index
q = Query(user_query)\
  .return_fields("full_text")
res = redis.ft("tweet:idx").search(q)
if res.total==0:
  print("No matches found")
else:
  res_df = pd.DataFrame([t.__dict__ for t in res.docs ]).drop(columns=["payload"])
  display(res_df)

No matches found


In [16]:
# a mesma parada, soh que melhor
# agora sim, usando um Vector Similarity contra o nosso Index
# notem que eu, malandramente, transformei a propria `user_query` em um vetor propio.
query_vector=text_to_embedding(user_query)
q = Query("*=>[KNN 10 @text_embedding $vector AS result_score]")\
                .return_fields("result_score","full_text")\
                .dialect(2)\
                .sort_by("result_score", True)
res = redis.ft("tweet:idx").search(q, query_params={"vector": query_vector})
#print(res)
res_df = pd.DataFrame([t.__dict__ for t in res.docs ]).drop(columns=["payload"])
res_df

,id,result_score,full_text
0,tweet:2907,0.61014932394,"Now in member chat: #Oil, #Energy sector, $XOM..."
1,tweet:2892,0.623246729374,"RT @philstockworld: Now in member chat: #Oil, ..."
2,tweet:5405,0.629442632198,https://t.co/3IJBXa5wuf Historic oil price plu...
3,tweet:5406,0.631321430206,Historic oil price plunge trashes sector's pro...
4,tweet:5676,0.632252633572,"I don't think the US can legally do that, and ..."
5,tweet:44,0.632612347603,https://t.co/9VjKMnpm7n\r\n\r\n#Cheniere Energ...
6,tweet:3568,0.632830858231,Top 200 energy stocks summary\r\n147 up\r\n48 ...
7,tweet:9402,0.639989078045,#OIL $SPY $CVS $XOM #companies $TSLA #TAX Scr...
8,tweet:5654,0.64328700304,..and oil still 25.74 LMAO &gt;&gt;&gt;NO DEM...
9,tweet:12459,0.643579721451,"@mintzmyer @tradewinds Fairness, you don’t fol..."
